<a href="https://colab.research.google.com/github/masteroojway/saidl_project/blob/multim/multimodality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/pbloem/kgbench-loader.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/pbloem/kgbench-loader.git to /tmp/pip-req-build-49ibj8vi
  Running command git clone --filter=blob:none --quiet https://github.com/pbloem/kgbench-loader.git /tmp/pip-req-build-49ibj8vi
  Resolved https://github.com/pbloem/kgbench-loader.git to commit 320e614c6df0fee0ab8806b40b1770b7fa1f4ca8
  Preparing metadata (setup.py) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
!pip install torch_geometric

In [ ]:
from transformers import AutoTokenizer, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch
import kgbench as kg
from torch_geometric.data import Data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_id = "google/paligemma-3b-mix-224"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16)
processor = PaliGemmaProcessor.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# Load dataset
data = kg.load('dmg777k', torch=True)

loaded data dmg777k (54.37s).


In [ ]:
#Data visualization:-
print("Total number of entities:- ", data.num_entities, sep='')
print("Total number of relations:- ", data.num_relations, sep='')
print("Total number of classes:- ", data.num_classes, sep='')

#knowledge graph :- connects related entities

print("Edges of the knowledge graph:-",data.triples.shape)
help(data.i2e) #function of i2e
help(data.e2i) #function of e2i
#dataset contains dutch monuments classified into 5 different classes. It
#contains 8399 different monuments with zero missing values

Total number of entities:- 341270
Total number of relations:- 60
Total number of classes:- 5
Edges of the knowledge graph:- torch.Size([777124, 3])
Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, v

In [ ]:
from PIL import Image

In [ ]:
#does not work
def extract_features(entity_id):
    try:
        # 1. Extract Dutch text description from tuple
        entity_info = data.i2e[entity_id]

        # The tuple structure appears to be: (description, language_tag)
        if isinstance(entity_info, tuple) and len(entity_info) >= 1:
            text_desc = entity_info[0]  # Get Dutch description text
        else:
            text_desc = str(entity_info)

        # Clean text while preserving Dutch special characters
        text_desc = text_desc.split('". ')[-1].replace('"', '').strip()

        # 2. Image handling (using dataset utilities)
        image = None
        try:
            # Find image datatype - common keys: 'image', 'afbeelding' (Dutch)
            img_dtypes = [k for k in data.datatype_g2l.keys()
                        if any(s in k.lower() for s in ['image', 'afbeelding'])]

            if img_dtypes:
                dtype = img_dtypes[0]
                local_idx = data.datatype_g2l[dtype][entity_id]  # Dictionary access
                images = data.get_images()
                if local_idx < len(images):
                    image = images[local_idx]
        except Exception as img_error:
            print(f"Image error for {entity_id}: {str(img_error)}")

        # 3. Process multilingual input (Dutch text + optional image)
        if image:
            inputs = processor(
                text=text_desc,  # PaLI-Gemma handles Dutch naturally
                images=image,
                return_tensors="pt"
            ).to(model.device, model.dtype)
        else:
            inputs = processor(
                text=text_desc,
                return_tensors="pt"
            ).to(model.device, model.dtype)

        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)

        return outputs.encoder_hidden_states[-1][:, 0, :].cpu().float()

    except Exception as e:
        print(f"Error processing entity {entity_id}: {str(e)}")
        return torch.zeros(1, 2048)  # Match hidden size

We can pass in our preprocessed inputs.

In [ ]:
# Batch feature extraction
node_features = []
for i in range(data.num_entities):
    features = extract_features(i)
    node_features.append(features)

node_features = torch.cat(node_features).to(torch.float32)

In [ ]:
#what neural net i would use if it worked
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
class gnn(torch.nn.Module):
    def __init__(self, in_dim = node_features.size(1), hidden_dim = 256, out_dim = data.num_classes):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.classifier = torch.nn.Linear(hidden_dim, out_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = F.elu(x)

        return self.classifier(x)
#Adam optimizer for that neural net
optimizer = torch.optim.Adam(gnn.parameters(), lr=0.001, weight_decay=5e-4)

In [ ]:
def test():
    gnn.eval()
    with torch.no_grad():
        logits = gnn(pyg_data)

    accs = []
    for mask in [pyg_data.train_mask, pyg_data.val_mask, pyg_data.test_mask]:
        pred = logits[mask].argmax(1)
        acc = (pred == pyg_data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs
def train():
    gnn.train()
    optimizer.zero_grad()
    out = gnn(pyg_data)
    loss = F.cross_entropy(out[pyg_data.train_mask], pyg_data.y[pyg_data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
best_val_acc = 0
temp = 10 #to stop when for 10 epochs val_acc has not increased (has saturated)
counter = 0

for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        counter = 0
        torch.save(gnn.state_dict(), 'best_model.pt')
    else:
        counter += 1

    if counter >= temp:
        print(f"Early stopping at epoch {epoch}")
        break

    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')